In [4]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
env = "staging"

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/rrm_radar_channel_stat/".format(env=env)

# date_day= datetime.datetime.today().strftime("%Y%m%d")
# hr = '23'
# date_day = "20200316"

date_day = "2020-04-*"
hr = '*'

rrm_local_path = "dt={day}/hr={hr}/*".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)




s3://mist-aggregated-stats-staging/aggregated-stats/rrm_radar_channel_stat/dt=2020-04-*/hr=*/*


In [7]:
# rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)
rdd_rrm_local = spark.sparkContext.textFile(rrm_local_path)
rdd_rrm_local.take(10)

# df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
# df_rrm.printSchema()

['site_id,ap_id,pre_channel,radar_channel_count',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefd61,124,1.0',
 'b7368ed6-379d-4e49-be60-a94ccc9e5702,5c5b353e1a1d,136,1.0',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefe06,120,1.0',
 '866808ba-1ff2-4e99-8e17-34ee1d3c70f4,5c5b35cf12a1,64,1.0',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefe4c,120,1.0',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefb40,60,1.0',
 '07574870-228c-4d0b-9d15-ae2f6beb8471,5c5b350e9796,104,1.0',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefc35,120,1.0',
 '934d5efe-5e35-48dd-99f9-5c57534708aa,5c5b35cefaff,120,1.0']

In [10]:
df_radar = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_local_path)
df_radar.createOrReplaceTempView("scan_data")
df_Schema = df_radar.schema

In [12]:
df_radar.show(5)

+--------------------+------------+-----------+-------------------+
|             site_id|       ap_id|pre_channel|radar_channel_count|
+--------------------+------------+-----------+-------------------+
|300f753a-3a01-4e6...|5c5b35ce12cf|         64|                1.0|
|e71abf65-870f-4b4...|5c5b35af6904|        104|                1.0|
|f9e819b5-628a-466...|5c5b358ebd3d|         52|                1.0|
|934d5efe-5e35-48d...|5c5b35cefd61|        124|                1.0|
|866808ba-1ff2-4e9...|5c5b35cf1bcf|        116|                1.0|
+--------------------+------------+-----------+-------------------+
only showing top 5 rows

